<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Market-Data/blob/main/USDT_M_Perp_Futures_symbols_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 需要資料:

symbol	string	否	交易對，例如：BTC...


recvWindow	int64	否	發起請求的時間戳，單位：毫秒

timestamp	int64	是	請求有效時間窗值，單位：毫秒

### 官方範例程式 2025/06/01

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 根網址
APIKEY = ""  # 使用者的 API 金鑰（需填入）
SECRETKEY = ""  # 使用者的秘密金鑰（需填入）

In [ ]:
def demo():
    payload = {}  # 請求的載荷，GET 請求通常為空
    path = '/openApi/swap/v2/quote/contracts'  # API 路徑
    method = "GET"  # 請求方法
    paramsMap = {}  # URL 參數字典，目前沒有額外參數
    paramsStr = parseParam(paramsMap)  # 將參數字典轉成 URL 查詢字串（包含 timestamp）
    return send_request(method, path, paramsStr, payload)  # 發送請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):
    # 使用 HMAC-SHA256 演算法，將 payload (URL 參數字串) 用 SECRETKEY 加密並取得簽名

    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名以供調試
    return signature  # 回傳簽名字串

In [ ]:
def send_request(method, path, urlpa, payload):
    # 建立完整 URL，並附加簽名參數 signature
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出完整請求 URL，方便除錯
    headers = {
        'X-BX-APIKEY': APIKEY,  # 將 APIKEY 加入請求標頭
    }

    # 發送 HTTP 請求，帶入 method、URL、標頭與載荷
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text  # 回傳 API 回應內容（字串）

In [ ]:
def parseParam(paramsMap):
    # 對參數鍵做排序，確保簽名一致性
    sortedKeys = sorted(paramsMap)

    # 將參數字典轉成 URL 查詢字串，如 key1=value1&key2=value2
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    if paramsStr != "":
        # 如果有其他參數，加上 timestamp
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 沒有參數則只有 timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行範例函式並印出結果